In [1]:
import sys
import os

# Get the directory of the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, ".."))

from modeling.rating import rating_get_df

rating_df = rating_get_df()

rating_df

,side,role,rating,season,week,team
0,def,qb1,-42.602151,2024,4,WSH
1,def,rb1,-20.877688,2024,4,WSH
2,def,rb2,-29.206989,2024,4,WSH
3,def,wr1,-25.913978,2024,4,WSH
4,def,wr2,-27.225806,2024,4,WSH
...,...,...,...,...,...,...
67671,off,rb2,18.500000,2015,1,ARI
67672,off,wr1,58.500000,2015,1,ARI
67673,off,wr2,52.178571,2015,1,ARI
67674,off,wr3,83.750000,2015,1,ARI


In [8]:
import pandas as pd

from modeling.compare import (
    compare_teams_rating,
)

from modeling.common import roles

from modeling.offense import offense_role_compute

from modeling.schedule import schedule_next_week

next_week = schedule_next_week()

season = next_week["season"]
week = next_week["week"] - 1

results = [
    compare_teams_rating(season, week, game["home"], game["away"])
    for game in next_week["games"]
]

game_results = pd.DataFrame([result[0] for result in results])

print("--games--")
print(game_results.sort_values(by=["diff"], ascending=False))

# element for row in my_array for element in row
teams_results = pd.concat([result[1] for result in results])


def role_lookup(season, week, team, role):
    team_week_role_df = offense_role_compute(
        {"season": season, "week": week, "team": team}
    )

    res = team_week_role_df.set_index("role").loc[role]

    return res["playerDisplay"]


print("--roles--")
for role in roles:
    print(f"--{role}--")
    role_df = teams_results[["team", role]].copy()

    role_df["player"] = role_df.apply(
        lambda row: role_lookup(season, week, row["team"], role), axis=1
    )

    print(role_df.sort_values(by=[role], ascending=False))

--games--
   winner loser home away        diff
3     MIA    NE  MIA   NE  230.228037
6     BUF   HOU  BUF  HOU  223.293338
11    PIT   DAL  DAL  PIT  113.831176
10    SEA   NYG  NYG  SEA  111.314664
12     NO    KC   NO   KC  107.401577
7     DEN    LV   LV  DEN   99.074603
8     ARI    SF  ARI   SF   60.189393
4     CLE   WSH  CLE  WSH   51.844691
2     CIN   BAL  BAL  CIN   39.166633
5     JAX   IND  IND  JAX   38.211274
1     CHI   CAR  CHI  CAR   34.267632
9      GB   LAR   GB  LAR   28.570299
0     ATL    TB  ATL   TB   12.067938
--roles--
--wr1--
  team         wr1            player
0   GB  134.291544  Dontayvion Wicks
0   NO  112.843548    Rashid Shaheed
1  HOU   97.901875      Nico Collins
1  JAX   97.832772    Christian Kirk
1  SEA   95.579301        DK Metcalf
1  DEN   92.855645  Courtland Sutton
1  CIN   89.123656       Tee Higgins
1   KC   88.768779      Travis Kelce
0  IND   81.072424        Josh Downs
0  DAL   75.722581       CeeDee Lamb
0  ATL   73.998656      Drake Lon

/var/folders/sx/xmtc134d0w520kgzwx2q0zsh0000gn/T/ipykernel_84975/1802316005.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_df["player"] = role_df.apply(
/var/folders/sx/xmtc134d0w520kgzwx2q0zsh0000gn/T/ipykernel_84975/1802316005.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_df["player"] = role_df.apply(
/var/folders/sx/xmtc134d0w520kgzwx2q0zsh0000gn/T/ipykernel_84975/1802316005.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

  team        wr3            player
1  LAR  83.631354          --fake--
0  BUF  76.828641     Khalil Shakir
1   KC  71.443996     Xavier Worthy
0  ATL  69.376344    Darnell Mooney
0  IND  62.287945   Adonai Mitchell
0   NO  57.825269     Juwan Johnson
0  CHI  56.434332      Keenan Allen
1  DEN  51.317765     Josh Reynolds
1  SEA  50.558519     Tyler Lockett
0  DAL  48.393935     Brandin Cooks
0   GB  48.330969       Jayden Reed
0  BAL  47.924800     Isaiah Likely
1   TB  45.893554      Chris Godwin
0  NYG  41.247849    Darius Slayton
1   NE  37.079301   DeMario Douglas
1  WSH  34.978495        Noah Brown
1  JAX  34.880568        Gabe Davis
0   LV  30.842204          --fake--
0  CLE  28.563172  Blake Whiteheart
1  CIN  28.173387     Erick All Jr.
0  MIA  20.806452     Tanner Conner
1  PIT  19.242473     Van Jefferson
1   SF  15.176075  Deebo Samuel Sr.
0  ARI  10.768548       Greg Dortch
1  CAR   0.805556    Jonathan Mingo
1  HOU -16.460484    Dalton Schultz
--rb1--
  team        rb1   

/var/folders/sx/xmtc134d0w520kgzwx2q0zsh0000gn/T/ipykernel_84975/1802316005.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_df["player"] = role_df.apply(
/var/folders/sx/xmtc134d0w520kgzwx2q0zsh0000gn/T/ipykernel_84975/1802316005.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_df["player"] = role_df.apply(
/var/folders/sx/xmtc134d0w520kgzwx2q0zsh0000gn/T/ipykernel_84975/1802316005.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [11]:
from modeling.common import fields as common_fields

from modeling.team import team_selector_decode

from modeling.stats import stats_rest_display


def stats_players(
    selector, players: list[str], include: bool = True, aggregate: bool = False
) -> pd.DataFrame:
    base = team_selector_decode(selector)

    if include:
        base = base[base["playerDisplay"].isin(players)]
    else:
        base = base[~base["playerDisplay"].isin(players)]

    if len(base.index) == 0:
        return None

    if aggregate:
        rtn_series = (
            base.groupby("week").agg({stat: "sum" for stat in common_fields}).mean()
        )

        rtn = pd.DataFrame([rtn_series])
        rtn["playerDisplay"] = stats_rest_display

        return rtn.set_index("playerDisplay").reset_index()
    else:
        rtn = base.groupby("playerDisplay").agg(
            {stat: "mean" for stat in common_fields}
        )

        if include:
            return rtn.reindex(index=players).reset_index()
        else:
            return rtn.reindex(index=base["playerDisplay"].unique()).reset_index()


df = team_selector_decode({"season": 2023, "week": 10, "team": "PHI"})

print(
    stats_players(
        df,
        players=["D'Andre Swift", "Kenneth Gainwell", "A.J. Brown", "DeVonta Smith"],
    )
)

print("==============")

print(
    stats_players(
        df,
        players=["D'Andre Swift", "Kenneth Gainwell", "A.J. Brown", "DeVonta Smith"],
        aggregate=True,
    )
)

print("==============")

print(
    stats_players(
        df,
        players=["D'Andre Swift", "Kenneth Gainwell", "A.J. Brown", "DeVonta Smith"],
        include=False,
    )
)

      playerDisplay  passCmp  passAtt  passYds  passTd  passInt  passLong  \
0     D'Andre Swift      0.0      0.0      0.0     0.0      0.0       0.0   
1  Kenneth Gainwell      0.0      0.0      0.0     0.0      0.0       0.0   
2        A.J. Brown      0.0      0.0      0.0     0.0      0.0       0.0   
3     DeVonta Smith      0.0      0.0      0.0     0.0      0.0       0.0   

   passRating  passTargetYds  rushAtt  ...    recCmp      recYds     recTd  \
0         0.0            0.0    15.00  ...  3.333333   18.444444  0.111111   
1         0.0            0.0     6.75  ...  1.875000    9.250000  0.000000   
2         0.0            0.0     0.00  ...  7.444444  111.666667  0.666667   
3         0.0            0.0     0.00  ...  4.666667   59.222222  0.444444   

   recDrops  recLong  recDepth  recYac  sacked   fumbles  fumblesLost  
0       0.0      0.0       0.0     0.0     0.0  0.333333     0.111111  
1       0.0      0.0       0.0     0.0     0.0  0.250000     0.125000  
2      